In [66]:
from glob import glob
from tqdm import tqdm
import json
import os
import pandas as pd
import numpy as np
import pickle
import sys
# import cv2
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Model

In [67]:
img_height, img_width = 224, 224  # 이미지 크기
batch_size = 16  # batch size 너무 크게하면 메모리 초과 뜸
data_loc = 'C:/Users/multicampus/projects/fresh/data' # your data loc

## 사과 양광 데이터로만 시도

In [27]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    f'{data_loc}/Training/apple_yanggwang',
    validation_split=0.2,
    subset="training",
    seed=123,  # random 값 고정
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 5544 files belonging to 3 classes.
Using 4436 files for training.


In [28]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  f'{data_loc}/Training/apple_yanggwang',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 5544 files belonging to 3 classes.
Using 1108 files for validation.


In [29]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    f'{data_loc}/Validation/apple_yanggwang',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 792 files belonging to 3 classes.


In [30]:
cls = train_ds.class_names
num_classes = len(cls)
print(cls)

['apple_yanggwang_L', 'apple_yanggwang_M', 'apple_yanggwang_S']


In [31]:
model = tf.keras.Sequential([
#   tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [32]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
  metrics=['accuracy'])

In [34]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=15
)

Epoch 1/15
278/278 [==============================] - 102s 366ms/step - loss: 0.6800 - accuracy: 0.6898 - val_loss: 0.3120 - val_accuracy: 0.8773
Epoch 2/15
278/278 [==============================] - 107s 382ms/step - loss: 0.3318 - accuracy: 0.8756 - val_loss: 0.2782 - val_accuracy: 0.8944
Epoch 3/15
278/278 [==============================] - 106s 380ms/step - loss: 0.2410 - accuracy: 0.9092 - val_loss: 0.1716 - val_accuracy: 0.9404
Epoch 4/15
278/278 [==============================] - 103s 366ms/step - loss: 0.2212 - accuracy: 0.9218 - val_loss: 0.2694 - val_accuracy: 0.9007
Epoch 5/15
278/278 [==============================] - 106s 378ms/step - loss: 0.1803 - accuracy: 0.9319 - val_loss: 0.2836 - val_accuracy: 0.8872
Epoch 6/15
278/278 [==============================] - 104s 373ms/step - loss: 0.1531 - accuracy: 0.9457 - val_loss: 0.1331 - val_accuracy: 0.9522
Epoch 7/15
278/278 [==============================] - 106s 378ms/step - loss: 0.1016 - accuracy: 0.9608 - val_loss: 0.1671 -

In [35]:
model.save('my_model_grade_224_1116.h5')

In [36]:
print('모델 테스트 데이터셋에 돌려본 결과')
print('test loss, test acc')
print(model.evaluate(test_ds))

모델 테스트 데이터셋에 돌려본 결과
test loss, test acc
50/50 [==============================] - 6s 116ms/step - loss: 3.0737 - accuracy: 0.6073
[3.0737197399139404, 0.6073232293128967]


In [25]:
# model = tf.keras.models.load_model("my_model_grade_1116.h5")
pred = []
y = []
for b_x, b_y in tqdm(test_ds):
    pred.extend(model.predict(b_x).tolist())
    y.extend(b_y.numpy().tolist())
pred = np.array(pred)
y = np.array(y)
pred_one = np.array([np.argmax(x) for x in pred])
acc = np.mean(pred_one == y)
print(acc)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [01:18<00:00,  1.57s/it]

0.5972222222222222


In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 224, 224, 16)      448       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 112, 112, 32)      4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 56, 56, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 28, 28, 32)       

## 사과 부사 데이터

In [69]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    f'{data_loc}/Training/apple_fuji',
    validation_split=0.2,
    subset="training",
    seed=123,  # random 값 고정
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  f'{data_loc}/Training/apple_fuji',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    f'{data_loc}/Validation/apple_fuji',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

cls = train_ds.class_names
num_classes = len(cls)
print('class names;', cls)

model = tf.keras.Sequential([
#   tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
  metrics=['accuracy'])

Found 7140 files belonging to 3 classes.
Using 5712 files for training.
Found 7140 files belonging to 3 classes.
Using 1428 files for validation.
Found 1020 files belonging to 3 classes.
class names; ['apple_fuji_L', 'apple_fuji_M', 'apple_fuji_S']


In [73]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=15
)

Epoch 1/15
357/357 [==============================] - 116s 324ms/step - loss: 0.8100 - accuracy: 0.7211 - val_loss: 0.2354 - val_accuracy: 0.9188
Epoch 2/15
357/357 [==============================] - 126s 351ms/step - loss: 0.2278 - accuracy: 0.9146 - val_loss: 0.2054 - val_accuracy: 0.9293
Epoch 3/15
357/357 [==============================] - 116s 324ms/step - loss: 0.1666 - accuracy: 0.9398 - val_loss: 0.4604 - val_accuracy: 0.8543
Epoch 4/15
357/357 [==============================] - 131s 364ms/step - loss: 0.1422 - accuracy: 0.9482 - val_loss: 0.2316 - val_accuracy: 0.9209
Epoch 5/15
357/357 [==============================] - 135s 375ms/step - loss: 0.1399 - accuracy: 0.9494 - val_loss: 0.1586 - val_accuracy: 0.9447
Epoch 6/15
357/357 [==============================] - 139s 389ms/step - loss: 0.1327 - accuracy: 0.9510 - val_loss: 0.1220 - val_accuracy: 0.9573
Epoch 7/15
357/357 [==============================] - 135s 377ms/step - loss: 0.1030 - accuracy: 0.9615 - val_loss: 0.0881 -

In [74]:
model.save('my_model_grade_224_1116_apple_fuji.h5')

In [75]:
print('모델 테스트 데이터셋에 돌려본 결과')
print('test loss, test acc')
print(model.evaluate(test_ds))

모델 테스트 데이터셋에 돌려본 결과
test loss, test acc
64/64 [==============================] - 7s 102ms/step - loss: 0.4519 - accuracy: 0.9176
[0.4519018232822418, 0.9176470637321472]
